# Checking the results of the assertion SLM approach

In [66]:
import pandas as pd


df = pd.read_csv('/drive2/kaggle/pii-dd/piidd/training/assertion/outputs/2024-04-17/07-23-31/20240417075501_preds.csv', engine='python', quotechar='"', sep=",", escapechar='\\', encoding='utf-8')

In [67]:
df = df.drop_duplicates(subset=['text']).dropna(axis=0)
df.shape

(2496, 15)

In [68]:
score_cols = ['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT',
       'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME',
       'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'O']

df["pred"] = df[score_cols].idxmax(axis=1)

(df["pred"] == df["label"]).mean()

0.9927884615384616

In [69]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

wrong = df[df["pred"] != df["label"]]
print(wrong.shape)

wrong

(18, 16)


,Unnamed: 0,text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,pred
151,151,"A systematic approach for problem solving\n\nI have used story telling tool to solve the problem .I preferred this tool because it adds more effectiveness. First let me tell you the story about a boy named <START>Stephen<END>, who is very poor and orphan. He used to work in a tooth paste manufacturing company to feed himself. He is well skilled and with in short time he was given a promotion as a supervisor for physical department.\n\nAll was going well for few years. The M.D has decided to increase the production. So, all of the board members have decided to attain a meeting for the analysis of how to increase the company income. All the team managers and supervisors had attended",B-NAME_STUDENT,1.433380e-06,3.547649e-06,0.000319,3.758058e-06,2.326605e-06,0.000018,0.000008,2.268275e-06,0.000035,2.021389e-06,2.202785e-06,0.999603,O
323,323,"a supervisor for physical department.\n\nAll was going well for few years. The M.D has decided to increase the production. So, all of the board members have decided to attain a meeting for the analysis of how to increase the company income. All the team managers and supervisors had attended the meeting. The objective of the meeting is “How to increase the company annual income”. All the people have referred to the past census to figure why the income has decreased.\n\n<START>Stephen<END> has gone through his survey and his reports were telling that the product has still maintained its demand in the market. Users also like that taste. He noticed their product and its dimensions of the mouth of the paste. He increased the gap of the tooth mouth by few millimeters. All the team members were agreed to this plan and had done a dramatic change in production. After that the sales have increased for the product. Finally the company had succeeded in increasing their production.",B-NAME_STUDENT,1.621426e-06,2.616459e-06,0.000108,3.187024e-06,1.464837e-06,0.000029,0.000006,2.264357e-06,0.000015,1.485001e-06,2.131324e-06,0.999827,O
416,416,"inhabit; we want to create holistic approaches. There’s no 100% solution. As professor Maii said in the beginning: “we’re all expected to get lean- to do more with less.” And so I reiterate, we can work smarter, not harder. Thank you for reading my work. You will find images of the tools I used below.\n\nPadlet Image: (It was too large, so I had to take a portion of it for display purposes)\n\nLuis Gonzales <START>Daniel<END> Nowak\n\nBuilding Block Castle\n\nJust a brief explanation: Empathy is the door/window into the castle, as it’s a necessary\n\ncomponent that eventually opens the way for the exploration of the other parts of the castle; hence the reasoning why understanding is the essentially the flag or the emblem that signifies that this is what our group is all about. The other figures are just as important, as they represent what others would like to see, their hopes; nevertheless, both empathy and understanding",B-NAME_STUDENT,6.985181e-07,1.987951e-06,0.000043,1.073467e-06,8.899352e-07,0.000003,0.000003,5.690015e-07,0.999524,1.443097e-06,1.356989e-06,0.000419,I-NAME_STUDENT
495,495,"Why Should I Mind Map?\n\nThe soul never thinks without a mental image.\n\n<START>Aristotle<END>\n\nThere are many reasons why mind mapping is used by millions of professionals, creatives and students all over the world. Here are the top 4:\n\nMind maps structure information.\n\nMind maps can store and structure vast amounts of information. They display hierarchy, show relationships between individual ideas and enable you to see the “big picture” at a glance. These features also make mind maps an ideal tool to present information to others, create knowledge pools and solve complex problems.\n\nMind",O,5.451398e-07,2.295124e-06,0.999651,1.323137e-06,1.934576e-06,0.000003,0.000009,1.257624e-06,0.000009

In [42]:
import gradio as gr




def update(idx, label, rnd, wrong):

    if rnd:
        filter_ = ~df.label.isna()
        if label:
            filter_ = filter_ & (df.label == label)
        if wrong:
            filter_ = filter_ & (df.label != df.pred)

        row = df[filter_].sample(n=1)
    else:
        row = df.iloc[idx]

    return row.text.tolist()[0], create_html_table(row)




def create_html_table(row):
    
    # Extract scores and sort them
    sorted_scores = sorted([(c, row.iloc[0][c]) for c in score_cols], key=lambda x: x[1], reverse=True)
    sorted_scores = [("Label", row['label'].tolist()[0])] + sorted_scores
    
    # Create HTML table
    html = "<table border='1'>"
    html += "<tr><th></th><th></th></tr>"
    for metric, score in sorted_scores:
        html += f"<tr><td>{metric}</td><td>{score}</td></tr>"
    html += "</table>"

    return html


with gr.Blocks() as demo:

    idx = gr.Slider()
    label = gr.Dropdown(choices=df.label.unique().tolist())
    rnd = gr.Checkbox(label="Random")
    wrong = gr.Checkbox(label="Is Wrong")
    get_new = gr.Button("New")

    text = gr.Textbox()

    scores = gr.HTML()


    get_new.click(
        update,
        inputs=[idx, label, rnd, wrong],
        outputs=[text, scores]
    )


demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [5]:
df.style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,Unnamed: 0,text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O
0,0,"created themed lunch days, and organized games and activities for students to enjoy during their lunch break. To track the progress of my efforts, I monitored the cafeteria's sales and foot traffic data. The data showed a gradual increase in the number of students and faculty members coming to the cafeteria for lunch, and the feedback I received was overwhelmingly positive. Many people commented on the improved quality and va name Felicia Partlow, contact 558.830.2276; alias breslin47; profile https://www.instagram.com/dempsey96 | jrhyne@uindy.edu 163 Amy Haven Dylanbury, MD 76821 riety of the food, as well as the more inviting atmosphere. However, there were still areas where I could have done better. For instance, I realized that I could have done a better job of promoting the new menu items and making sure that everyone was aware of the changes. I also could have done more to accommodate dietary restrictions, such as gluten-free and nut-free options. In the future, I would approach this challenge differently by being more proactive in communicating the changes",I-STREET_ADDRESS,0.010805,0.015256,0.011132,0.022438,0.009143,0.014141,0.009964,0.012651,0.013480,0.046502,0.824268,0.010220
1,1,"and seasoned geothermal engineer, who had previously employed Lean Six Sigma with considerable success in his past roles. Inspired by Danai Gurira's portrayal of a strong-willed problem solver, I decided to embark on this strategic approach with determination and resilience. The Application Process of the Chosen Strategy The initial phase of the Lean Six Sigma methodology involved defining the problem and establishing project boundaries. I collaborated with my team to determine the specific KP Laurencia Penner; wimmer@rutgers.edu Pin No: A.u. 317880, page https://www.facebook.com/cecelia_steffen | 487 Motuara Mews Jamesneath 6044 Is (Key Performance Indicators) and laid out our improvement objectives, such as reducing downtime and increasing energy output while respecting environmental constraints. Next, we measured the existing performance of the geothermal systems by collecting and analyzing relevant data, such as system pressure, temperature, and flow rates. This data-driven approach helped us identify specific areas where improvements could be made and set benchmarks for measuring progress. Analyzing the data led us to pinpoint several issues affecting the system's efficiency",B-STREET_ADDRESS,0.036031,0.045358,0.019599,0.061223,0.036004,0.049817,0.049068,0.037686,0.035829,0.239467,0.365857,0.024060
2,2,"role in my understanding and application of Design Thinking. Her guidance and feedback helped me to effectively adapt the methodology to the specific context of the problem at hand. Lastly, I would like to mention that I am inspired by Lakeith Stanfield, an American actor known for his role in ""Atlanta"" where he plays a character that navigates the complexities of the digital world in a unique and authentic way. He embodies th Bazile Cloyd, https://www.instagram.com/clarke53, 77622 montée Dufour Lac- Marcel, SK V8C2P9 Number 3120691952, loraperalta - email: adaline_oreilly@mit.edu e spirit of creativity and adaptability that is crucial in the consulting industry. Thank you for considering my perspective, Bazile Cloyd",B-USERNAME,0.193909,0.043916,0.032564,0.044695,0.063551,0.090451,0.215899,0.031841,0.039859,0.080376,0.130121,0.032819
3,3,"contributed to the success of our client meetings and overall strategy implementation. The SWOT analysis proved to be highly effective in addressing the volatile market challenges. It enabled us to develop tailored investment strategies addressing each client's unique needs. The strategy's successes included improved diversification, increased returns, and strengthened client relationships, while its limitations primarily involved 